In [ ]:
%load_ext autoreload
%autoreload 1

import sys
sys.path.append("../utils/")

import pdb

import threading
from queue import Queue
import requests
import bs4
import time
import signal

from WikiMultiQuery import *
from GraphAPI import GraphCreator

%aimport WikiMultiQuery
%aimport GraphAPI

In [ ]:
gc = GraphCreator("Prevention science")
nodes = list(gc.graph.nodes)
len(nodes)

In [ ]:
gc.expand_network_th(threads=10)
len(gc.graph.nodes)

In [ ]:
def handle_alarm(signum, frame):
    raise RuntimeError

signal.signal(signal.SIGALRM, handle_alarm)
def test(gc):
    try:
        signal.alarm(2)
        gc.expand_network_th()
    except:
        signal.alarm(0)
        return
    
test(gc)

In [ ]:
def timeout(article_list):
    return len(article_list) / 100 * 5 # 5 seconds for every hundred articles

timeout(list(range(225)))

In [ ]:
class Query:
    
    def __init__(self):
        self.count = 0
        def handle_alarm(signum, frame):
            raise RuntimeError

        signal.signal(signal.SIGALRM, handle_alarm)
        
    def query_wrapper(self, article_titles):
        article_queue = Queue()

        def get_url(article_title):
            try:
                result = wiki_multi_query([article_title])
                self.count += 1
                print(self.count)
            except:
                print(f"SKIP - {article_title}")

        def process_queue():
            while True:
                current_article = article_queue.get()
                get_url(current_article)
                article_queue.task_done()

        for i in range(10): # 20 threads
            t = threading.Thread(target=process_queue)  
            t.daemon = True
            t.start()

        for title in article_titles:
            article_queue.put(title)

        article_queue.join()



In [ ]:
q = Query()
try:
    signal.alarm(int(timeout(nodes)))
    q.query_wrapper(nodes)
    signal.alarm(0)
except:
    signal.alarm(0)
    print(q.count)